In [ ]:
import model
import keras

import pandas as pd
import crystal_loader
from tqdm import tqdm
# import symmetry
from symmetry import *


fpath = "./TiO2/structure0001.xsf"

df = pd.read_csv(fpath)

structs, labels = crystal_loader.get_dataset("./TiO2", halt=-1)

In [ ]:
import dill

test_params = ((0, 0.003, 1, 1), (0, 0.005, 2, 1), (0, 0.02, 1, -1))
tio_params_ang = {("Ti", "Ti"): test_params, ("Ti", "O"): test_params, ("O", "O"): test_params}

tio_params_rad = {"Ti": ((0, 0.003), (0, 0.01)), "O": ((0, 0.003), (0, 0.01))}

layers = [keras.layers.Dense(50, activation="tanh"), keras.layers.Dense(20, activation="tanh")]

MLP1 = model.MLP(n_inputs=13, layers=layers, output_func="linear")

MLP1.compile(
    optimizer = keras.optimizers.Adam(learning_rate=0.0001),
    loss = keras.losses.MeanAbsoluteError(),
    metrics = [keras.metrics.RootMeanSquaredError()]
)

with open("TiO2_features.dill", "rb") as dill_file:
    features = dill.load(dill_file)

In [ ]:
import tensorflow as tf
from sklearn.model_selection import train_test_split

F = tf.ragged.constant(features, ragged_rank=1)
print(F.shape)

Xtrain, Xval, ytrain, yval = F[:300,:,:], F[300:,:,:], labels[:300], labels[300:]

In [ ]:
import tensorflow as tf

In [ ]:
bpnn = MLP1.fit(
    Xtrain, ytrain,
    batch_size = 20,
    epochs = 40,
    validation_data = (Xval, yval),
    verbose = 10
)